# Example usage of Superset API client

This notebook demonstrates how to export and import dashboards from superset using the `superset_io` package.

In [ ]:
%load_ext autoreload
%autoreload 2

import logging

logging.basicConfig(level="INFO")

## Using programmatically

If you need fine control or scripting capabilities, you can use the `SupersetApiSession` and `SuperSetApiClient` classes directly in your Python code. These allow to interact with the Superset API for exporting, importing and browsing dashboards.

In [ ]:
from superset_io.api import SuperSetApiClient, SupersetApiSession

# Authenticate
session = SupersetApiSession.from_credentials(
    base_url="http://localhost:8088",
    username="admin",
    password="admin",
)

# Create API client
client = SuperSetApiClient(session=session)

### Listing dashboards

You can list available dashboards using the `get_dashboards` method.

In [ ]:
# Untyped for now
client.get_dashboards()

### Exporting a Dashboard

You can export a dashboard by specifying its ID, this returns a zipfile object.

In [ ]:
from pathlib import Path

zip_bytes, zip_file = client.export_dashboard(dashboard_id=1)

# Save the exported dashboard to a file
# Note this will overwrite existing files
# consider using a unique filename
with Path("../exported_dashboard/dashboard.zip").open("wb") as f:
    f.write(zip_bytes)
zip_file.extractall("../exported_dashboard")

### Importing a Dashboard

Similar to the export, the import requires a zipfile containing the dashboard data.

In [ ]:

from superset_io.utils import (
    validate_assets_bundle_structure,
    zipfile_buffer_from_folder,
)

# Zip folder back into a zipfile for import
zipfile_buffer = zipfile_buffer_from_folder(
    "../exported_dashboard/assets_export_20260212T092634"
)

# The time stamp and folder name do not matter
zipfile_buffer = zipfile_buffer_from_folder("../exported_dashboard/assets")

# Alternative: open existing zipfile
# zipfile_buffer = zipfile_buffer_from_zipfile("../exported_dashboard/dashboard.zip")

validate_assets_bundle_structure(zipfile_buffer)


# with zipfile.ZipFile(zipfile_buffer) as zf:
#     pprint(zf.namelist())

In [6]:
res = client.import_dashboard(
    zipfile_buffer=zipfile_buffer,
)
res.status_code

DEBUG:urllib3.connectionpool:http://localhost:8088 "POST /api/v1/assets/import/ HTTP/1.1" 200 17


200

In [7]:
print(res.request.headers.get("Content-Type"))
print(res.text)
print(res.json())

multipart/form-data; boundary=77300cb7429af503f11076914504f3da
{"message":"OK"}

{'message': 'OK'}
